In [1]:
using ProgressMeter
using Random #, Distributions
using PyPlot

In [3]:
mutable struct PTParams{T}
    TMax       :: T
    λ          :: T
    NTemps     :: Int
    Nexchanges :: Int
    PTParams{T}() where {T} = new()
end;

mutable struct MetropolisParams{T}
    NSteps       :: Int
    StepSize     :: T
    GaussianStep :: Bool
    MetropolisParams{T}() where {T} = new()
end;

# data structure for the points of the phase space

x is a tensor with NTemp walkers, each one with d dimensions. The structure will be:

$x[temp, dim]$

In [ ]:
function parallel_tempering(PTParameters::PTParams, MetropolisParams::MetropolisParams, initial_guess)

    # Define energy function

    Ener(x)          = -5*exp(-(x+3)^2) -exp(-2*(x-5.0)^2)

    # -------------------------------------------------------
    # Local variables
    # -------------------------------------------------------
    
    NTemps           = PTParameters.NTemps
    Nexchanges       = PTParameters.Nexchanges
    TMax             = PTParameters.TMax
    NSteps           = MetropolisParams.NSteps
    StepSize         = MetropolisParams.StepSize

    λ                = PTParameters.λ
    temperatures     = zeros(NTemps)
    temperatures[1]  = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]

    xo               = initial_guess
    xn               = zeros(size(xo))

    Eveco           = Ener.(xo)
    TupEBest        = findmin(Eveco)
    EBest           = TupEBest[1]
    EBestPos        = xo[TupEBest[2]]

    display("Initial guess: ", xo)
    display("Initial energies: ", Eveco)
    display("Initial best energy: ", EBest[1])
    display("Temperatures: ", temperatures')
    # debug energy and init guess
    is_plot = false
    if is_plot
        clf()
        plot(-10:10, Ener.(-10:10), label="Energy")
        plot(xo, Ener.(xo), "ro", label="Initial guess")
        plot(EBestPos, EBest[1], "go", label="Best guess")
        title("Initial guess")
        xlabel("x")
        ylabel("Energy")
        legend()
        # if ./out exists, save the figure there
        if isdir("./out")
            savefig("./out/energy")
        else
            show()
        end
    end

    # -------------------------------------------------------
    # Parallel tempering loop
    # -------------------------------------------------------
    @showprogress for _ in 1:PTParameters.Nexchanges+1
        # Metropolis step
        for i in 1:NSteps
            xn = xo .+ StepSize*randn(NTemps) # generate new positions
            Evecn = Ener.(xn) # compute the energy of each new position of each chain (NTemp long vector)
            ΔE_vec = Evecn .- Eveco # compute the energy difference between the new and old positions

            mask1 = ΔE_vec .< 0 # if the new position is better, accept it
            mask2 = !mask1 & (rand(NTemps) .< exp.(-ΔE_vec./temperatures)) # if the new position is worse, metropolis probability
            mask = mask1 .| mask2 # combine the two masks

            xo = xn .* mask .+ xo .* .!mask #

            Eveco = Ener.(xo) # compute the energy of each updated (accepted/rejected) position of each chain (NTemp long vector)
            prob_best_guess = findmin(Eveco) # from the new sampled energies, find the best guess
            if prob_best_guess[1] < EBest # compare the new best guess with the old one. If its better, update it
                # EBest: energy, position
                # EBestPos: position
                EBest    = prob_best_guess[1]
                EBestPos = xo[prob_best_guess[2]]
            end
        end

        # Exchange step
        # no check for the lowest state, as we don't explore space here
        for temp in 1:NTemps-1
            ΔE_exchange = (Ener(xo[temp]) - Ener(xo[temp+1])) * (1/temperatures[temp] - 1/temperatures[temp+1])
            if ΔE_exchange < 0
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
                # Temp dependency is put inside ΔE_exchange!!!!!!
            elseif rand() < exp(-ΔE_exchange)
                xo[temp], xo[temp+1] = xo[temp+1], xo[temp]
            end
        end
    end

    return xo, EBest, EBestPos
end;